In [1]:
##Import
import pickle
import pandas as pd
import itertools
import math

In [2]:
#pull in dicts 
pre_outcomes = pickle.load(open("pickle_pre.p","rb"))  #note: these don't need to be different files, just for convenience
post_outcomes = pickle.load(open("pickle_post.p","rb"))   #note: these don't need to be different files, just for convenience
#key
samples = pd.read_excel("gp17_pre_post_key.xlsx")[["condition","pre_file","post_file"]]

In [3]:
#get enrichment/depletion
fewer_outcomes = 'wpn'
rel_abundance_dict = {}
for i in samples.index:
    rel_abundance_dict[samples['condition'][i]] = {}
    for product in itertools.product(fewer_outcomes, repeat=4):
        if pre_outcomes[samples['pre_file'][i]][product] == float(0) and post_outcomes[samples['post_file'][i]][product] < float(0.01):
            rel_abundance_dict[samples['condition'][i]][product] = 'undetermined'        
        elif pre_outcomes[samples['pre_file'][i]][product] < float(0.01) and post_outcomes[samples['post_file'][i]][product] == float(0):
            rel_abundance_dict[samples['condition'][i]][product] = 'undetermined'      
        elif pre_outcomes[samples['pre_file'][i]][product] == float(0) and post_outcomes[samples['post_file'][i]][product] >= float(0.01):
            rel_abundance_dict[samples['condition'][i]][product] = '>= 100X enriched'
        elif pre_outcomes[samples['pre_file'][i]][product] >= float(0.01) and post_outcomes[samples['post_file'][i]][product] == float(0):
            rel_abundance_dict[samples['condition'][i]][product] = '>= 100X depleted'        
        else:
            rel_abundance_dict[samples['condition'][i]][product] = post_outcomes[samples['post_file'][i]][product] / pre_outcomes[samples['pre_file'][i]][product]
            

In [4]:
#get enrichment/depletion normalized to the full wt
fewer_outcomes = 'wpn'
rel_abundance_dict_norm_wt = {}
for i in samples.index:
    wt_relative_abundance = post_outcomes[samples['post_file'][i]][('w','w','w','w')] / pre_outcomes[samples['pre_file'][i]][('w','w','w','w')]
    rel_abundance_dict_norm_wt[samples['condition'][i]] = {}
    for product in itertools.product(fewer_outcomes, repeat=4):
        if pre_outcomes[samples['pre_file'][i]][product] == float(0) and post_outcomes[samples['post_file'][i]][product] < float(0.01):
            rel_abundance_dict_norm_wt[samples['condition'][i]][product] = 'undetermined'        
        elif pre_outcomes[samples['pre_file'][i]][product] < float(0.01) and post_outcomes[samples['post_file'][i]][product] == float(0):
            rel_abundance_dict_norm_wt[samples['condition'][i]][product] = 'undetermined'      
        elif pre_outcomes[samples['pre_file'][i]][product] == float(0) and post_outcomes[samples['post_file'][i]][product] >= float(0.01):
            rel_abundance_dict_norm_wt[samples['condition'][i]][product] = '>= 100X enriched'
        elif pre_outcomes[samples['pre_file'][i]][product] >= float(0.01) and post_outcomes[samples['post_file'][i]][product] == float(0):
            rel_abundance_dict_norm_wt[samples['condition'][i]][product] = '>= 100X depleted'   
        else:
            rel_abundance_dict_norm_wt[samples['condition'][i]][product] = (post_outcomes[samples['post_file'][i]][product] / pre_outcomes[samples['pre_file'][i]][product])/wt_relative_abundance


In [5]:
# Convert dictionary to DataFrame
df = pd.DataFrame(rel_abundance_dict)

# Output DataFrame to Excel file
df.to_excel("rel_abundance.xlsx", index=True)

In [6]:
# Convert dictionary to DataFrame
df = pd.DataFrame(rel_abundance_dict_norm_wt)

# Output DataFrame to Excel file
df.to_excel("rel_abundance_norm_wt.xlsx", index=True)

In [7]:
#for all doubles, get difference from sum of singles (in log10)
epi_dict = {}
for rep in rel_abundance_dict_norm_wt:
    epi_dict[rep] = {}
    for outcome in rel_abundance_dict_norm_wt[rep]:
        singles = []
        try:
            if outcome[0] is not 'w':
                singles.append(math.log(rel_abundance_dict_norm_wt[rep][('%s' %outcome[0], 'w', 'w', 'w')],10))
            if outcome[1] is not 'w':
                singles.append(math.log(rel_abundance_dict_norm_wt[rep][('w', '%s' %outcome[1], 'w', 'w')],10))
            if outcome[2] is not 'w':
                singles.append(math.log(rel_abundance_dict_norm_wt[rep][('w', 'w', '%s' %outcome[2], 'w')],10))       
            if outcome[3] is not 'w':
                singles.append(math.log(rel_abundance_dict_norm_wt[rep][('w', 'w', 'w', '%s' %outcome[3])],10))
            if rel_abundance_dict[rep][outcome] is not 'undetermined':
                epi_dict[rep][outcome] = math.log(rel_abundance_dict_norm_wt[rep][outcome],10) - sum(singles)
            else: epi_dict[rep][outcome] = 'undetermined'
        except TypeError:
            epi_dict[rep][outcome] = 'no single'

In [8]:
# Convert dictionary to DataFrame
df = pd.DataFrame(epi_dict)

# Output DataFrame to Excel file
df.to_excel("epi_log10_norm_wt.xlsx", index=True)